# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集
- 訓練一個 CNN 模型來學習門牌號碼資料集。
***

### 本課程範例實際在GPU執行時，失敗無效。

### 解決方式參考@ magikerwin1993的做法如下
#### 參考來源: https://github.com/magikerwin1993/1st-DL-CVMarathon/blob/master/homework/Day-033/julia_033_hw.ipynb

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux.Optimise: Optimiser, WeightDecay
using Flux: onehotbatch, onecold, logitcrossentropy
using Statistics, Random
import ProgressMeter
import MLDatasets
using CUDAapi

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1273
┌ Warning: `haskey(::TargetIterator, name::String)` is deprecated, use `Target(; name=name) !== nothing` instead.
│   caller = llvm_compat(::VersionNumber) at compatibility.jl:176
└ @ CUDAnative C:\Users\Administrator\.julia\packages\CUDAnative\e0IdN\src\compatibility.jl:176
┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1273


## Set the parameters

In [2]:
η = 5e-4             # learning rate
λ = 5e-4             # L2 regularizer param, implemented as weight decay
epochs = 60          # number of epochs
batchsize = 256      # batch size

256

# Using CUDA

In [3]:
use_cuda = CUDAapi.has_cuda_gpu()
if use_cuda
    device = gpu
    println("Training on GPU")
else
    device = cpu
    println("Training on CPU")
end

Training on GPU


# Read the datasets

In [4]:
function get_data(batchsize=256)
    xtrain, ytrain = MLDatasets.SVHN2.traindata(Float32, 1:20000)
    xtest , ytest  = MLDatasets.SVHN2.testdata(Float32, 1:2000)

    #xtrain = reshape(xtrain, 28, 28, 1, :)
    #xtest  = reshape(xtest, 28, 28, 1, :)

    ytrain = onehotbatch(ytrain, 1:10)
    ytest  = onehotbatch(ytest, 1:10)

    train_loader = DataLoader(xtrain, ytrain, batchsize=batchsize, shuffle=true)
    test_loader  = DataLoader(xtest, ytest,  batchsize=batchsize)
    
    return train_loader, test_loader
end

train_loader, test_loader = get_data(batchsize);

# CNN Model

In [5]:
# write your model here
num_params(model) = sum(length, Flux.params(model))

function buildModel(imgsize=(32,32,3))

    return Chain(
        x -> reshape(x, imgsize..., :),
        Conv((5, 5), 3=>24, stride=2),
        BatchNorm(24, relu),
        Conv((1, 1), 24=>24, stride=1),
        BatchNorm(24, relu),
        Conv((3, 3), 24=>48, stride=2),
        BatchNorm(48, relu),
        Conv((1, 1), 48=>48, stride=1),
        BatchNorm(48, relu),
        Conv((3, 3), 48=>96, stride=2),
        BatchNorm(96, relu),
        Conv((1, 1), 96=>96, stride=1),
        BatchNorm(96, relu),
        flatten,
        Dense(384, 10),
        softmax
    )
end

model = buildModel() |> device;
println("CNNs model: $(num_params(model)) trainable params");

CNNs model: 70594 trainable params


In [6]:
fake_input = randn((32, 32, 3)) |> device
fake_output = model(fake_input)

┌ Warning: `Target(triple::String)` is deprecated, use `Target(; triple=triple)` instead.
│   caller = ip:0x0
└ @ Core :-1


10×1 CuArrays.CuArray{Float32,2,Nothing}:
 0.103127114
 0.09932483 
 0.09783187 
 0.09790238 
 0.106007904
 0.10220131 
 0.10514694 
 0.093225464
 0.09879361 
 0.09643855 

# Loss function

In [7]:
loss(ŷ, y) = logitcrossentropy(ŷ, y)

round4(x) = round(x, digits=4)

function calc_loss_accuracy(loader, model, device)
    l = 0f0
    acc = 0
    ntot = 0
    for (x, y) in loader
        x, y = x |> device, y |> device
        ŷ = model(x)
        l += loss(ŷ, y) * size(x)[end]        
        acc += sum(onecold(ŷ |> cpu) .== onecold(y |> cpu))
        ntot += size(x)[end]
    end
    return (loss = l/ntot |> round4, acc = acc/ntot*100 |> round4)
end

calc_loss_accuracy (generic function with 1 method)

# Callback Function

In [8]:
# callback function
function callback(epoch)
    train = calc_loss_accuracy(train_loader, model, device)
    test = calc_loss_accuracy(test_loader, model, device)        
    println("Epoch: $epoch   Train: $(train)   Test: $(test)")
end

callback (generic function with 1 method)

# Training

In [9]:
# define optimizer
opt = ADAM(η) 
if λ > 0 
    opt = Optimiser(opt, WeightDecay(λ))
end

Optimiser(Any[ADAM(0.0005, (0.9, 0.999), IdDict{Any,Any}()), WeightDecay(0.0005)])

In [10]:
params_model = Flux.params(model)

println("Starting Training")
callback(0)

for epoch in 1:epochs
    progress = ProgressMeter.Progress(length(train_loader))
    
    # update parameters
    for (x, y) in train_loader
        x = x |> device
        y = y |> device
        grads_model = Flux.gradient(params_model) do
            ŷ = model(x)
            loss(ŷ, y)
        end
        Flux.Optimise.update!(opt, params_model, grads_model)
        ProgressMeter.next!(progress)   # comment out for no progress bar
    end
    
    #  logging
    if epoch % 5 == 0
        callback(epoch)
    end
end

Starting Training
Epoch: 0   Train: (loss = 2.3023f0, acc = 8.055)   Test: (loss = 2.3023f0, acc = 7.9)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:46
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 5   Train: (loss = 1.6657f0, acc = 83.155)   Test: (loss = 1.7397f0, acc = 74.55)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 10   Train: (loss = 1.5936f0, acc = 89.585)   Test: (loss = 1.6838f0, acc = 79.9)

...

Epoch: 50   Train: (loss = 1.5065f0, acc = 96.935)   Test: (loss = 1.6439f0, acc = 83.3)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 55   Train: (loss = 1.5568f0, acc = 92.48)   Test: (loss = 1.6945f0, acc = 78.55)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 60   Train: (loss = 1.5637f0, acc = 92.0)   Test: (loss = 1.6836f0, acc = 79.8)


# Evaluation

In [11]:
test = calc_loss_accuracy(test_loader, model, device)        
println("Test: $(test)")

Test: (loss = 1.6836f0, acc = 79.8)
